In [1]:
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
con = sqlite3.connect('prod.db')

In [3]:
df = pd.read_sql_query("""
SELECT l.match_id
,  l.account_id
,  fp.name
, dp.role
, l.kills
, l.deaths
,l.assists
, l.last_hits
, l.denies
, l.gold_per_min
, l.teamfight_participation
, l.stuns
, l.tower_kills
, l.roshan_kills
, l.firstblood_claimed
, l.camps_stacked
, l.rune_pickups
, l.obs_placed
from logs_player_match_stats as l 
JOIN dim_players as dp
ON dp.account_id = l.account_id
JOIN f_players as fp
ON l.account_id = fp.account_id
WHERE fp.team_tag in ('Alliance', 'OG','EG', 'UND', 'IG', 'VP')
""",
 con).fillna(0)

In [4]:
def make_fantasy_points(df):
    df['FP'] = df['kills']*.3 \
                    + 3 + df['deaths']*-.3 \
                    + (df['last_hits']+df['denies'])*0.003\
                    + df['gold_per_min']*0.002\
                    + df['teamfight_participation']*3\
                    + df['stuns']*0.05\
                    + df['tower_kills'] \
                    + df['roshan_kills'] \
                    + df['firstblood_claimed']*4 \
                    + df['camps_stacked']*0.5 \
                    + df['rune_pickups']*0.25 \
                    + df['obs_placed']*0.5
    for col in df.columns[4:]:
        df[f'{col}_rolling'] = df[col].rolling(window=3).mean()
        df[f'{col}_final'] = df[f'{col}_rolling'].tail(1).values[0]
    return df

In [5]:
df = df.groupby('account_id').apply(make_fantasy_points)

In [6]:
df

,match_id,account_id,name,role,kills,deaths,assists,last_hits,denies,gold_per_min,...,firstblood_claimed_rolling,firstblood_claimed_final,camps_stacked_rolling,camps_stacked_final,rune_pickups_rolling,rune_pickups_final,obs_placed_rolling,obs_placed_final,FP_rolling,FP_final
0,5979057906,12231202,LIMMP,2,2,3,4,323,3,474,...,NaN,0.333333,NaN,2.333333,NaN,15.666667,NaN,1.0,NaN,20.871747
1,5979128923,12231202,LIMMP,2,7,0,12,492,6,641,...,NaN,0.333333,NaN,2.333333,NaN,15.666667,NaN,1.0,NaN,20.871747
2,5979186071,12231202,LIMMP,2,4,7,12,315,5,435,...,0.0,0.333333,2.000000,2.333333,13.333333,15.666667,1.666667,1.0,15.378314,20.871747
3,5989792474,12231202,LIMMP,2,4,3,13,332,9,454,...,0.0,0.333333,1.333333,2.333333,15.666667,15.666667,1.666667,1.0,16.398932,20.871747
4,5989881522,12231202,LIMMP,2,6,1,7,200,2,499,...,0.0,0.333333,1.333333,2.333333,16.333333,15.666667,1.333333,1.0,15.964197,20.871747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,6156066114,480412663,gpk~,2,3,6,3,217,10,421,...,0.0,0.000000,3.000000,1.000000,9.666667,7.000000,1.000000,2.0,15.968090,14.278079
1605,6156190097,480412663,gpk~,2,2,4,3,375,3,561,...,0.0,0.000000,3.333333,1.000000,10.000000,7.000000,0.666667,2.0,13.328683,14.278079
1606,6157433167,480412663,gpk~,2,6,5,3,346,15,554,...,0.0,0.000000,2.666667,1.000000,7.333333,7.000000,1.000000,2.0,12.610222,14.278079
1607,6157557980,480412663,gpk~,2,13,0,12,240,15,592,...,0.0,0.000000,2.666667,1.000000,9.333333,7.000000,1.666667,2.0,15.428236,14.278079


In [7]:
def drop_cols(df):
    for col in df.columns[4:]:
        if not col.endswith('_final'):
            df.drop(col, axis = 1, inplace = True)
    return df

In [8]:
df = drop_cols(df)

In [9]:
df.drop('match_id', axis = 1, inplace = True)

In [10]:
df.drop_duplicates(inplace = True)

In [11]:
df.reset_index(inplace=True, drop = True)

In [12]:
df

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
0,12231202,LIMMP,2,9.666667,3.000000,7.666667,307.333333,6.000000,550.333333,0.738889,73.954945,1.000000,0.000000,0.333333,2.333333,15.666667,1.000000,20.871747
1,18180970,Handsken,4,3.333333,5.666667,11.666667,115.666667,1.000000,361.000000,0.634524,52.679545,0.333333,0.000000,0.333333,5.333333,2.666667,3.666667,14.742882
2,19672354,N0tail,4,1.666667,5.666667,14.333333,60.666667,5.333333,301.666667,0.506285,42.287047,0.333333,0.000000,0.000000,7.666667,5.000000,9.333333,16.317873
3,25907144,Cr1t-,4,2.333333,7.000000,10.666667,35.333333,3.666667,240.000000,0.690341,42.166322,0.333333,0.000000,0.000000,0.666667,3.333333,6.000000,10.876339
4,38628747,MoonMeander,4,3.333333,8.000000,12.000000,89.000000,4.666667,305.333333,0.581197,13.015025,0.000000,0.000000,0.333333,3.666667,2.666667,2.333333,9.886008
5,41231571,s4,3,2.000000,4.333333,12.333333,212.666667,5.666667,422.333333,0.620317,34.296108,0.333333,0.000000,0.000000,1.666667,2.000000,0.333333,9.208758
6,56351509,DM,3,3.000000,5.000000,9.333333,181.000000,14.000000,403.000000,0.476309,34.750610,0.333333,0.333333,0.333333,0.333333,1.666667,0.000000,9.540792
7,84772440,iceiceice,3,3.000000,6.333333,4.666667,240.000000,9.333333,421.666667,0.423927,6.337012,0.666667,0.000000,0.000000,0.666667,3.000000,0.000000,6.929964
8,86745912,Arteezy,1,0.666667,4.333333,8.666667,425.333333,20.000000,611.333333,0.551768,0.000000,2.666667,0.333333,0.000000,4.000000,3.333333,0.000000,11.947303
9,88271237,Ceb,3,6.333333,5.666667,15.333333,327.666667,3.000000,567.666667,0.728758,2.763053,1.000000,0.000000,0.000000,1.000000,4.000000,0.333333,10.318427


In [13]:
df.sort_values(by = ['role', 'FP_final'])

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
23,168028715,flyfly,1,4.666667,4.333333,4.000000,264.666667,7.666667,508.333333,0.456043,12.378333,1.333333,0.333333,0.000000,1.333333,1.666667,0.000000,9.670712
8,86745912,Arteezy,1,0.666667,4.333333,8.666667,425.333333,20.000000,611.333333,0.551768,0.000000,2.666667,0.333333,0.000000,4.000000,3.333333,0.000000,11.947303
13,97658618,Timado,1,6.333333,3.000000,9.333333,419.333333,9.000000,648.333333,0.603276,33.447105,2.000000,0.666667,0.000000,1.333333,3.333333,0.000000,14.230518
17,124801257,Nightfall,1,7.333333,2.333333,5.666667,434.666667,18.666667,721.000000,0.561554,8.976146,3.666667,0.333333,0.666667,2.333333,6.333333,0.000000,18.852136
16,111620041,SumaiL,1,9.666667,2.666667,9.333333,403.666667,25.000000,706.666667,0.585721,0.000000,4.333333,1.333333,0.333333,0.666667,7.000000,0.666667,18.973164
28,412753955,Nikobaby龙的传人,1,8.000000,2.666667,8.333333,438.333333,9.333333,681.666667,0.702302,24.860423,4.333333,1.333333,0.333333,0.666667,4.333333,0.000000,19.072926
11,94054712,Topson,2,7.666667,5.333333,14.666667,178.000000,8.333333,480.666667,0.758270,34.737930,0.000000,0.333333,0.000000,1.666667,8.666667,0.333333,12.732041
29,480412663,gpk~,2,7.666667,3.000000,8.333333,275.333333,18.666667,542.666667,0.643314,34.616065,0.666667,0.333333,0.000000,1.000000,7.000000,2.000000,14.278079
22,154715080,Abed,2,7.666667,3.666667,3.000000,366.333333,8.333333,551.666667,0.647727,26.569598,0.000000,1.333333,0.000000,2.000000,10.333333,1.000000,15.115662
25,202217968,Emo,2,8.666667,2.000000,6.000000,246.666667,7.000000,524.000000,0.770565,5.684350,0.666667,0.000000,0.000000,2.000000,17.666667,0.333333,15.654913


In [14]:
df_core = df[df['role'].isin(['1','3']) ]

In [15]:
df_mid  = df[df['role'] == '2' ]

In [16]:
df_sup = df[df['role'] == '4' ]

In [20]:
df_core.sort_values(by = 'FP_final', ascending=False)

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
28,412753955,Nikobaby龙的传人,1,8.000000,2.666667,8.333333,438.333333,9.333333,681.666667,0.702302,24.860423,4.333333,1.333333,0.333333,0.666667,4.333333,0.000000,19.072926
16,111620041,SumaiL,1,9.666667,2.666667,9.333333,403.666667,25.000000,706.666667,0.585721,0.000000,4.333333,1.333333,0.333333,0.666667,7.000000,0.666667,18.973164
17,124801257,Nightfall,1,7.333333,2.333333,5.666667,434.666667,18.666667,721.000000,0.561554,8.976146,3.666667,0.333333,0.666667,2.333333,6.333333,0.000000,18.852136
13,97658618,Timado,1,6.333333,3.000000,9.333333,419.333333,9.000000,648.333333,0.603276,33.447105,2.000000,0.666667,0.000000,1.333333,3.333333,0.000000,14.230518
8,86745912,Arteezy,1,0.666667,4.333333,8.666667,425.333333,20.000000,611.333333,0.551768,0.000000,2.666667,0.333333,0.000000,4.000000,3.333333,0.000000,11.947303
18,126212866,SabeRLighT,3,5.666667,5.000000,14.333333,296.666667,11.666667,513.333333,0.819801,23.427404,2.333333,0.000000,0.000000,0.000000,1.666667,0.000000,11.532439
9,88271237,Ceb,3,6.333333,5.666667,15.333333,327.666667,3.000000,567.666667,0.728758,2.763053,1.000000,0.000000,0.000000,1.000000,4.000000,0.333333,10.318427
19,138857296,JT-,3,3.333333,4.000000,6.666667,216.666667,7.000000,442.666667,0.523684,18.424704,0.666667,0.333333,0.333333,0.666667,2.333333,0.000000,10.098621
23,168028715,flyfly,1,4.666667,4.333333,4.000000,264.666667,7.666667,508.333333,0.456043,12.378333,1.333333,0.333333,0.000000,1.333333,1.666667,0.000000,9.670712
6,56351509,DM,3,3.000000,5.000000,9.333333,181.000000,14.000000,403.000000,0.476309,34.750610,0.333333,0.333333,0.333333,0.333333,1.666667,0.000000,9.540792


In [21]:
df_mid.sort_values(by = 'FP_final', ascending=False)

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
0,12231202,LIMMP,2,9.666667,3.000000,7.666667,307.333333,6.000000,550.333333,0.738889,73.954945,1.000000,0.000000,0.333333,2.333333,15.666667,1.000000,20.871747
26,238239590,Bryle,2,8.000000,4.000000,8.333333,314.666667,9.333333,522.333333,0.673789,54.820339,0.333333,0.000000,0.333333,2.333333,10.666667,0.666667,16.812384
25,202217968,Emo,2,8.666667,2.000000,6.000000,246.666667,7.000000,524.000000,0.770565,5.684350,0.666667,0.000000,0.000000,2.000000,17.666667,0.333333,15.654913
22,154715080,Abed,2,7.666667,3.666667,3.000000,366.333333,8.333333,551.666667,0.647727,26.569598,0.000000,1.333333,0.000000,2.000000,10.333333,1.000000,15.115662
29,480412663,gpk~,2,7.666667,3.000000,8.333333,275.333333,18.666667,542.666667,0.643314,34.616065,0.666667,0.333333,0.000000,1.000000,7.000000,2.000000,14.278079
11,94054712,Topson,2,7.666667,5.333333,14.666667,178.000000,8.333333,480.666667,0.758270,34.737930,0.000000,0.333333,0.000000,1.666667,8.666667,0.333333,12.732041


In [26]:
df_sup

,account_id,name,role,kills_final,deaths_final,assists_final,last_hits_final,denies_final,gold_per_min_final,teamfight_participation_final,stuns_final,tower_kills_final,roshan_kills_final,firstblood_claimed_final,camps_stacked_final,rune_pickups_final,obs_placed_final,FP_final
1,18180970,Handsken,4,3.333333,5.666667,11.666667,115.666667,1.000000,361.000000,0.634524,52.679545,0.333333,0.0,0.333333,5.333333,2.666667,3.666667,14.742882
2,19672354,N0tail,4,1.666667,5.666667,14.333333,60.666667,5.333333,301.666667,0.506285,42.287047,0.333333,0.0,0.000000,7.666667,5.000000,9.333333,16.317873
3,25907144,Cr1t-,4,2.333333,7.000000,10.666667,35.333333,3.666667,240.000000,0.690341,42.166322,0.333333,0.0,0.000000,0.666667,3.333333,6.000000,10.876339
4,38628747,MoonMeander,4,3.333333,8.000000,12.000000,89.000000,4.666667,305.333333,0.581197,13.015025,0.000000,0.0,0.333333,3.666667,2.666667,2.333333,9.886008
10,94049589,Fng,4,0.333333,5.000000,11.333333,75.666667,4.666667,281.333333,0.486270,6.466809,0.000000,0.0,0.000000,7.666667,3.666667,13.333333,15.602483
12,94155156,Fly,4,2.333333,5.666667,8.000000,53.666667,9.000000,255.000000,0.622790,38.657024,0.000000,0.0,0.000000,3.666667,2.666667,10.000000,13.999222
14,103039499,super,4,0.333333,5.333333,10.333333,20.333333,1.333333,239.000000,0.563636,12.818522,0.000000,0.0,0.000000,4.000000,3.333333,6.333333,10.374835
15,103735745,Saksa,4,4.000000,6.666667,16.666667,56.666667,9.000000,312.666667,0.688286,127.932203,0.333333,0.0,0.000000,1.000000,2.333333,7.333333,16.567134
20,139876032,kaka,4,1.666667,6.666667,8.333333,46.000000,2.000000,244.333333,0.510721,49.695684,0.000000,0.0,0.333333,4.666667,1.666667,8.333333,14.399615
21,145550466,DuBu,4,1.333333,6.666667,9.000000,18.000000,2.333333,224.666667,0.459402,41.903203,0.000000,0.0,0.000000,2.666667,1.666667,16.333333,15.300365
